In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import pandas as pd
import pymongo
from pymongo import MongoClient
import json
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv
import chardet

with open("/content/drive/My Drive/Colab Notebooks/data.csv", mode="rb") as ifile:
    # may need to increase byte count from 32 for better accuracy
    raw_bytes = ifile.read(64)
    encoding_name = chardet.detect(raw_bytes)["encoding"]
    

In [ ]:
encoding_name

'ascii'

In [ ]:
import os
os. getcwd()

'/content'

In [ ]:
df=pd.read_csv("/content/drive/My Drive/Colab Notebooks/data.csv", encoding='ascii',quotechar='"', delimiter=',')
df = df.iloc[: , 1:]
df=df.dropna()

NameError: ignored

In [ ]:
df

,spam,commentCount,dislikeCount,likeCount,viewCount,defaultAudioLanguage,likeToViews,dislikeToViews,commentToViews
0,0,1313,67,8732,123464,en,7.072507,0.054267,1.063468
1,0,1343,104,11478,213989,en,5.363827,0.048601,0.627602
2,0,1538,130,10866,250419,en,4.339128,0.051913,0.614171
3,0,2054,175,12464,271252,en,4.594989,0.064516,0.757229
4,0,2862,136,15737,283538,en,5.550226,0.047965,1.009389
...,...,...,...,...,...,...,...,...,...
79,1,0,0,12,137,en,8.759124,0.000000,0.000000
80,1,225,0,108,674,en,16.023739,0.000000,33.382789
81,1,3,0,5,57,US,8.771930,0.000000,5.263158
82,1,9,0,20,275,en,7.272727,0.000000,3.272727


In [ ]:
train, test = train_test_split(df, test_size=0.25)

In [ ]:
y_train=train.pop("spam")

In [ ]:
y_test=test.pop("spam")

In [ ]:
feature_columns=[]

for item in df.iloc[:,1:].columns:
  if df[item].dtype !="O":
    feature_columns.append(tf.feature_column.numeric_column(item,dtype=tf.float32))
  else:
    vocabulary = dftrain[item].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(item,vocabulary))



In [ ]:
feature_columns

[NumericColumn(key='commentCount', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='dislikeCount', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='likeCount', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='viewCount', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 VocabularyListCategoricalColumn(key='defaultAudioLanguage', vocabulary_list=('en', 'US', 'GB', 'hi'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 NumericColumn(key='likeToViews', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='dislikeToViews', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='commentToViews', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function




In [ ]:
train_input_fn = make_input_fn(train, y_train)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

In [ ]:
train_input_fn

<function __main__.make_input_fn.<locals>.input_function>

In [ ]:
linear_est=tf.estimator.LinearClassifier(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp38gqnl0t', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
linear_est.train(train_input_fn)


INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp38gqnl0t/model.ckpt-80
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 80...
INFO:tensorflow:Saving checkpoints for 80 into /tmp/tmp38gqnl0t/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 80...
INFO:tensorflow:loss = 23.304405, step = 80
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 100...
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmp38gqnl0t/model.ckpt.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 100...
INFO:tensorflow:Loss for final step: 76.225975.


In [ ]:
result=linear_est.evaluate(test_input_fn)

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-09-11T02:45:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp38gqnl0t/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.79790s
INFO:tensorflow:Finished evaluation at 2021-09-11-02:45:43
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.8095238, accuracy_baseline = 0.52380955, auc = 0.85, auc_precision_recall = 0.8937651, average_loss = 17.212, global_step = 100, label/mean = 0.47619048, loss = 17.212, precision = 1.0, prediction/mean = 0.2863466, recall = 0.6
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: /tmp/tmp38gqnl0t/model.ckpt-100


In [ ]:
print(result["accuracy"])
print(result)

0.8095238
{'accuracy': 0.8095238, 'accuracy_baseline': 0.52380955, 'auc': 0.85, 'auc_precision_recall': 0.8937651, 'average_loss': 17.212, 'label/mean': 0.47619048, 'loss': 17.212, 'precision': 1.0, 'prediction/mean': 0.2863466, 'recall': 0.6, 'global_step': 100}


In [ ]:
df

,spam,commentCount,dislikeCount,likeCount,viewCount,defaultAudioLanguage,likeToViews,dislikeToViews
0,0,1313,67,8732,123464,en,0.0708,0.0005
1,0,1343,104,11478,213989,en,0.0536,0.0005
2,0,1538,130,10866,250419,en,0.0434,0.0005
3,0,2054,175,12464,271252,en,0.0460,0.0006
4,0,2862,136,15737,283538,en,0.0555,0.0005
...,...,...,...,...,...,...,...,...
663,1,0,0,12,137,en,0.0876,0.0000
664,1,225,0,108,674,en,0.1603,0.0000
665,1,3,0,5,57,US,0.0877,0.0000
666,1,9,0,20,275,en,0.0728,0.0000
